In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
metrics = pd.read_csv("metrics.csv")

# Add index column
metrics["index"] = metrics.index
metrics

In [ ]:
# Plotting the data, with the x-axis being the number of epochs and the y-axis being the scores of reward, audio_sim and transcription_sim
# fig, ax = plt.subplots()
# ax.set(xlabel="Epochs", ylabel="Scores")
plt.title("Scores over epochs", fontsize=20)

sns.set_theme(style="whitegrid", palette="deep")

plt.xlabel("Epochs")
plt.locator_params(axis='x', integer=True)

data = metrics.melt('index', var_name='score', value_name='value')
sns.lineplot(x='index', y='value', hue='score', data=data)

In [ ]:
DELAY = 100


fig, axes = plt.subplots(3, 1, figsize=(15, 8), sharex=True)
fig.suptitle('MODEL DDPG Training Results')

metrics = metrics[0:]
sns.lineplot(ax=axes[0], x="index", y="reward", data=metrics, color="r")
reward_mean = [np.mean(metrics["reward"][i-DELAY:i]) for i in range(DELAY, len(metrics))]
sns.lineplot(ax=axes[0], x=range(DELAY, len(metrics)), y=reward_mean, color="black")
axes[0].set_ylabel("Reward")

sns.lineplot(ax=axes[1], x="index", y="transcription_sim", data=metrics, color="g")
transcription_mean = [np.mean(metrics["transcription_sim"][i-DELAY:i]) for i in range(DELAY, len(metrics))]
sns.lineplot(ax=axes[1], x=range(DELAY, len(metrics)), y=transcription_mean, color="black")
axes[1].set_ylabel("Transcription Similarity")

sns.lineplot(ax=axes[2], x="index", y="audio_dissim", data=metrics, color="b")
audio_mean = [np.mean(metrics["audio_dissim"][i-DELAY:i]) for i in range(DELAY, len(metrics))]
sns.lineplot(ax=axes[2], x=range(DELAY, len(metrics)), y=audio_mean, color="black")
axes[2].set_ylabel("Audio Dissimilarity")

axes[2].set_xlabel("Iteration")

# Align y-axis labels
fig.align_ylabels()